In [16]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

df = pd.read_csv("../data/Data_Repuestos_2018_2021.csv", 
                       header=1, 
                       usecols=[' noPeriodo',' idArticulo','MoImporte',' moPrecio', 'CnSalida 2018', ' caDescProduct', 'Factura o Boleta'],
                       )
product = pd.read_csv("../data/producto.csv")
df.head()

,noPeriodo,idArticulo,CnSalida 2018,moPrecio,MoImporte,Factura o Boleta,caDescProduct
0,20180100,25319,1.0,15.00,15.0,2018-1-159166- BOL,FOCO H11-12 100-90W CURVO
1,20180100,4923,1.0,23.00,23.0,2018-1-159167- BOL,FILTRO AIRE HYUNDAI TUCSON
2,20180100,25853,1.0,13.00,13.0,2018-1-159169- BOL,FILTRO GASOLINA NISSAN METAL GA15-16 INJECTADO
3,20180100,13497,1.0,90.00,90.0,2018-1-159171- BOL,JGO CABLES BUJIA KIA RIO LARGO
4,20180100,23900,4.0,6.25,25.0,2018-1-159171- BOL,BUJIA NGK


In [17]:
df.columns = [colum.strip() for colum in df.columns.tolist()]
product.columns = [colum.strip() for colum in product.columns.tolist()]

### Seleccionando solo los productos más relevantes

In [18]:
products = product['idArticulo'].unique().tolist()

In [19]:
df = df.loc[df['idArticulo'].isin(products)]
#df['Factura o Boleta'] = df['Factura o Boleta'].astype('category')
df['caDescProduct'] = df['caDescProduct'].str.strip()
#df['caDescProduct'] = df['caDescProduct'].astype('category')
df.dropna(axis=0, subset=['Factura o Boleta'], inplace=True)
df['Factura o Boleta'] = df['Factura o Boleta'].astype('str')
df.head()

,noPeriodo,idArticulo,CnSalida 2018,moPrecio,MoImporte,Factura o Boleta,caDescProduct
4,20180100,23900,4.0,6.25,25.0,2018-1-159171- BOL,BUJIA NGK
6,20180100,10964,1.0,60.00,60.0,2018-1-159173- BOL,REFRIGERANTE PRESTONE 50/50 USA
11,20180100,9950,1.0,10.00,10.0,2018-1-159176- BOL,REFRIGERANTE COOLANT AZUL/NATURAL TRANSP. GALO...
17,20180100,23900,4.0,7.00,28.0,2018-1-159179- BOL,BUJIA NGK
34,20180100,17414,4.0,7.50,30.0,2018-1-159194- BOL,BUJIA 5/8 R/L 2E DENSO 3194


In [20]:
basket = (df[['Factura o Boleta', 'idArticulo', 'CnSalida 2018']]
          .groupby(['Factura o Boleta', 'idArticulo'])['CnSalida 2018']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Factura o Boleta'))
basket.head()

idArticulo,551,594,1208,1224,1627,3745,3882,3883,4070,4827,...,20301,20305,20309,20310,20697,20769,21577,23900,26158,26170
Factura o Boleta,,,,,,,,,,,,,,,,,,,,,
2018- BB01-1017- BOL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018- BB01-1025- BOL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018- BB01-1037- BOL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018- BB01-1042- BOL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018- BB01-1044- BOL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
    return 0

basket_sets = basket.applymap(encode_units)


## Generando conjuntos de articulos frecuentes entre si [Link](https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/#:~:text=a%20frequent%20itemset%20is%20defined%20as%20a%20set%20of%20items%20that%20occur%20together%20in%20at%20least%2050%25%20of%20all%20transactions%20in%20the%20database.)


* Estamos usando un soporte del 7% de sensibilidad

In [50]:
frequent_itemsets = apriori(basket_sets, min_support=0.002, use_colnames=True)

In [51]:
frequent_itemsets.head()

,support,itemsets
0,0.029876,(551)
1,0.014938,(594)
2,0.026971,(1208)
3,0.017012,(1224)
4,0.037344,(1627)


## Generamos las reglas con su respectivo soporte, relación y relevancia. [link](https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/#:~:text=support%20is%20used%20to%20measure%20the%20abundance%20or%20frequency%20(often%20interpreted%20as%20significance%20or%20importance)%20of%20an%20itemset%20in%20a%20database.)

* Por qué la métrica **lift** ?[link](https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/#:~:text=The%20lift%20metric%20is%20commonly%20used%20to%20measure%20how%20much%20more%20often%20the%20antecedent%20and%20consequent%20of%20a%20rule%20A-%3EC%20occur%20together%20than%20we%20would%20expect%20if%20they%20were%20statistically%20independent.%20If%20A%20and%20C%20are%20independent%2C%20the%20Lift%20score%20will%20be%20exactly%201.)
* 1 quiere decir que son productos independientes, superior a 1 que contienen cierta relación

In [52]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(1627),(551),0.037344,0.029876,0.008437,0.225926,7.562243,0.007321,1.253271
1,(551),(1627),0.029876,0.037344,0.008437,0.282407,7.562243,0.007321,1.341507
2,(5579),(551),0.025450,0.029876,0.022268,0.875000,29.288194,0.021508,7.760996
3,(551),(5579),0.029876,0.025450,0.022268,0.745370,29.288194,0.021508,3.827326
4,(17779),(551),0.100000,0.029876,0.003181,0.031812,1.064815,0.000194,1.002000


In [53]:
rules.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   antecedents         70 non-null     object 
 1   consequents         70 non-null     object 
 2   antecedent support  70 non-null     float64
 3   consequent support  70 non-null     float64
 4   support             70 non-null     float64
 5   confidence          70 non-null     float64
 6   lift                70 non-null     float64
 7   leverage            70 non-null     float64
 8   conviction          70 non-null     float64
dtypes: float64(7), object(2)
memory usage: 5.0+ KB


In [59]:
results = pd.DataFrame.from_dict({"antecedents": [list(item)[0] for item in rules.antecedents.tolist()],
                                  "consequents":[list(item)[0] for item in rules.consequents.tolist()]})

In [60]:
results.head()

,antecedents,consequents
0,1627,551
1,551,1627
2,5579,551
3,551,5579
4,17779,551


In [61]:
results.to_csv('../data/basket_analysis.csv', index=False)